## Solución del problema

### 1.Instalación e importación de librerias

In [73]:
"""# Manejo de gráficas
%pip install scikit-plot
# Manejo de gráficas
%pip install scikit-plot
#Manejo de lemmatization spanish
%pip install stanza
#!{sys.executable} -m pip install pandas-profiling
"""
#Solo ejecutar si no se tiene instalado

'# Manejo de gráficas\n%pip install scikit-plot\n# Manejo de gráficas\n%pip install scikit-plot\n#Manejo de lemmatization spanish\n%pip install stanza\n#!{sys.executable} -m pip install pandas-profiling\n'

In [74]:
# librería Natural Language Toolkit, usada para trabajar con textos
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ascas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
#El lemmatizer de NLTK NO funciona en español, por lo que se usará el de Stanza
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
# ¿Qué es el lema de una palabra? ¿Qué tan dificil puede ser obtenerlo, piensa en el caso en que tuvieras que escribir la función que realiza esta tarea?
#nltk.download('wordnet')
import stanza
stanza.download('es') # descarga el modelo de lematización en español


2024-04-20 17:17:36 INFO: Downloaded file to C:\Users\ascas\stanza_resources\resources.json
2024-04-20 17:17:36 INFO: Downloading default packages for language: es (Spanish) ...
2024-04-20 17:17:38 INFO: File exists: C:\Users\ascas\stanza_resources\es\default.zip
2024-04-20 17:17:44 INFO: Finished downloading models and saved to C:\Users\ascas\stanza_resources


In [76]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

2024-04-20 17:17:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-20 17:17:44 INFO: Downloaded file to C:\Users\ascas\stanza_resources\resources.json
2024-04-20 17:17:45 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| pos       | ancora_charlm   |
| lemma     | ancora_nocharlm |

2024-04-20 17:17:45 INFO: Using device: cpu
2024-04-20 17:17:45 INFO: Loading: tokenize
2024-04-20 17:17:45 INFO: Loading: mwt
2024-04-20 17:17:45 INFO: Loading: pos
2024-04-20 17:17:46 INFO: Loading: lemma
2024-04-20 17:17:46 INFO: Done loading processors!


In [77]:

import pandas as pd
import numpy as np
import sys

#Usados en el proyecto 1
from nltk.tokenize import WordPunctTokenizer 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
######################################

import re, string, unicodedata

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

from sklearn.base import BaseEstimator, ClassifierMixin
from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

### 2. Perfilamiento y entendimiento de los datos

In [78]:
# Uso de la libreria pandas para la lectura de archivos
data=pd.read_csv('train_reviews.csv', sep=',', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data_train=data

In [79]:
data_train

,Review,Class
0,Muy buena atención y aclaración de dudas por p...,5
1,Buen hotel si están obligados a estar cerca de...,3
2,"Es un lugar muy lindo para fotografías, visite...",5
3,Abusados con la factura de alimentos siempre s...,3
4,Tuvimos un par de personas en el grupo que rea...,3
...,...,...
7870,"Me parece buen sistema, agiliza el transporte,...",4
7871,"Fue una escapada de un día desde el complejo, ...",4
7872,La Plaza de la Revolución es un lugar emblemát...,3
7873,Es la segunda ocasión que me quedo en los cuar...,1


In [80]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7875 entries, 0 to 7874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  7875 non-null   object
 1   Class   7875 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.2+ KB


#### 2. Entendimiento de los datos

In [81]:
from scipy import stats as st

textos = data_train.copy()
textos_pipeline = data_train.copy()
textos['Conteo'] = [len(x) for x in textos['Review']]

#Por ahora: La moda no da información relevante
"""def moda(textos):
    for i in textos['Review']: 
        dict = {}
        for x in i.split(' '): 
            print(i)
        
            if x in dict.keys():
                dict[x] += 1
            else:
                dict[x] = 1

        max_key = max(dict, key=dict.get)
        print(max_key)


moda(textos)
"""

#Max tiene el máximo tamaño de la palabra
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Review']]
#Max tiene el minimo tamaño de la palabra
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Review']]


def frecuenciaPalabras(texto):
    frecuenciaPalabras = {}

    for i in texto: 
        for x in i.split(' '): 
            if x in frecuenciaPalabras.keys():
                frecuenciaPalabras[x] += 1
            else:
                frecuenciaPalabras[x] = 1

    print(frecuenciaPalabras)
    return frecuenciaPalabras

def moda(texto):

    dict = {}
    for x in texto.split(' '):
    
        if x in dict.keys():
            dict[x] += 1
        else:
            dict[x] = 1

    max_key = max(dict, key=dict.get)
    return str(max_key)


textos['moda'] = [moda(i) for i in textos['Review']]



In [82]:
"""#Codigo para guardar los lemas en csv
textos.to_csv('sinProcesar.csv')"""


"#Codigo para guardar los lemas en csv\ntextos.to_csv('sinProcesar.csv')"

Se definen rutinas de preprocesamiento que se recopilan en la rutina preprocessing() y softPreprocessing(), la segunda rutina se aplica previo a la aplicación de lemmatización con el fin de obtener mejores resultados y tras lemmatizar si se aplica preprocessing()

In [83]:
spanish_stopwords = stopwords.words('spanish')
print(spanish_stopwords)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words
    

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

#def replace_numbers(words):
#    """Replace all interger occurrences in list of tokenized words with textual representation"""
#    p = inflect.engine()
#    print(words)
#    new_words = []
#    for word in words:
#        if word.isdigit():
#            new_word = p.number_to_words(word)
#            new_words.append(new_word)
#            print("if " + new_word)
#        else:
#            new_words.append(word)
#    return new_words

def remove_numbers(textos):
    textos = textos.replace(r'\d+,\d+', '', regex=True)
    textos = textos.replace(r'\d+', '', regex=True)
    textos = textos.replace(r'\d+.\d+', '', regex=True)
    return textos

def remove_stopwords(words):
    own = ['ser','haber','tener','ir','poder','hacer','hotel','pedir','llegar','mas','habitacion','habitación',
           'decir','lugar','dia','alguno','dar','querer','comida','ver','vista','despue',
           'un','dos','tres','cuatro','cuc','adema','buen']
    new_words = []
    for word in words:
        if word not in spanish_stopwords and word not in own:
            new_words.append(word)
    return new_words

def preprocessing(words):
    words = words.apply(to_lowercase)
 #   words = replace_numbers(words)
    words = words.apply(remove_punctuation)
#    words = remove_non_ascii(words)
    words = words.apply(remove_stopwords)
    #Pasar a str de una
    words = words.apply(getString)
    return words

def getString(list): 
    string = ""
    for w in list: 
        string += w + " "
    return string

def getTokens(textos): 
    textos=textos.str.split()
    return textos


def softPreprocessing(textos):
    textos =remove_numbers(textos)
    tokens = getTokens(textos)
    words = tokens.apply(to_lowercase)
    words = words.apply(remove_non_ascii)
    words = words.apply(getString)
    return words

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [84]:
from tqdm.notebook import tqdm
tqdm.pandas()
def lemmatizer(review):
    
    doc  =  nlp(review)
    #print (review)
    lemma = [[word.lemma for word in sent.words]  for sent in doc.sentences]
    finalLemma =[]
    for sent in lemma:
        for word in sent:  
            finalLemma.append(word)
    
    finalLemma = getString(finalLemma)

    return finalLemma

def applyLemmatizer(textos):

    return textos.progress_apply(lemmatizer) 


"""
textos['tokens']= textos.apply(lemmatizer,axis=1) #Aplica la lematización

#Codigo para guardar los lemas en csv
textos.to_csv('lemaSoftProcessing.csv')"""


"\ntextos['tokens']= textos.apply(lemmatizer,axis=1) #Aplica la lematización\n\n#Codigo para guardar los lemas en csv\ntextos.to_csv('lemaSoftProcessing.csv')"

In [85]:

def tokenizar(textos):
    textos = textos.apply(WordPunctTokenizer().tokenize)
    return textos
    

#### Pipeline 
Fuera del pipeline -< Dividir x train, xtest>
1. Eliminar numeros -> str del texto
2. 
2.0 cambiar a tokens
2.1 Aplicar softPreprocessing
    Minusculas
    Non ascii
    -> str del texto

3. Aplicar lemmatizer
    Transformar lista a str
4. Tokenizar
5. Aplicar preProcessing hard
6. Pasar a str otra vez
7. Hacer tfidf
8. Hacer modelo




In [86]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import FunctionTransformer

textos_pipeline.drop_duplicates(keep='first', inplace=True)

softPreprocessingTransformer = FunctionTransformer(softPreprocessing)
tokenizarTransformer = FunctionTransformer(tokenizar)
preprocessingTransformer = FunctionTransformer(preprocessing)
applyLemmatizerTransformer = FunctionTransformer(applyLemmatizer)

x_train, x_test = train_test_split(textos_pipeline, test_size=0.20, random_state=1) 
X_train, X_test, y_train, y_test = x_train['Review'], x_test['Review'],x_train['Class'], x_test['Class']
print(len(X_train),len(X_test),len(y_train),len(y_test))
"""('lemmatizer',applyLemmatizer),"""
pipe1 = Pipeline([('softPreprocessing',softPreprocessingTransformer),('tokenizer', tokenizarTransformer),('preprocessing',preprocessingTransformer),  ('tfidf', TfidfVectorizer()),('svc',SVC(C=10,gamma=1,kernel='rbf'))])

6241 1561 6241 1561


In [87]:
"""pipe.fit(X_train, y_train)"""

'pipe.fit(X_train, y_train)'

In [88]:
"""pipe.fit(X_train, y_train).score(X_test, y_test)"""

'pipe.fit(X_train, y_train).score(X_test, y_test)'

In [89]:
"""filename = "model.joblib"""

'filename = "model.joblib'

In [90]:
"""dump(pipe,filename)"""

'dump(pipe,filename)'

In [91]:
"""['model.joblib']"""

"['model.joblib']"

In [92]:
"""pipeline_loaded = load(filename)"""

'pipeline_loaded = load(filename)'

In [93]:
x_train, x_test = train_test_split(textos_pipeline, test_size=0.20, random_state=1) 
X_train, X_test, y_train, y_test = x_train['Review'], x_test['Review'],x_train['Class'], x_test['Class']
print(len(X_train),len(X_test),len(y_train),len(y_test))

softPreprocessingTransformer = FunctionTransformer(softPreprocessing)
tokenizarTransformer = FunctionTransformer(tokenizar)
preprocessingTransformer = FunctionTransformer(preprocessing)
applyLemmatizerTransformer = FunctionTransformer(applyLemmatizer)
""""""
pipe2 = Pipeline([('softPreprocessing',softPreprocessingTransformer),('lemmatizer',applyLemmatizerTransformer),('tokenizer', tokenizarTransformer),('preprocessing',preprocessingTransformer),  ('tfidf', TfidfVectorizer()),('svc',SVC(C=10,gamma=1,kernel='rbf'))])

6241 1561 6241 1561


In [94]:
pipe2.fit(X_train, y_train)

  0%|          | 0/6241 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
pipe2.score(X_test, y_test)

0.48110185778347214

In [ ]:
filename = "modelLemmatizerFinal.joblib"


In [ ]:
dump(pipe2,filename)

In [ ]:
"""from cloudpickle import dump
import cloudpickle
import funciones

cloudpickle.register_pickle_by_value(funciones)  

with open(filename, "wb") as pkl_file:
        dump(pipe2, pkl_file)"""

In [ ]:
['modelLemmatizerFinal.joblib']

['modelLemmatizerPruebaClase.joblib']

In [ ]:
pipeline_loaded2 = load("modelLemmatizerFinal.joblib")

### 6. Recursos utilizados: 

https://medium.com/escueladeinteligenciaartificial/procesamiento-de-lenguaje-natural-stemming-y-lemmas-f5efd90dca8

https://neptune.ai/blog/vectorization-techniques-in-nlp-guide

https://gitlab.virtual.uniandes.edu.co/ISIS3301/practicas/blob/master/ProcesamientoTextos/Preparaci%C3%B3n_de_textos_estudiante.ipynb
